In [32]:
from datasets import load_from_disk
import torch
from diffusers import AutoencoderKL
from PIL import Image
import numpy as np

In [9]:
vae = AutoencoderKL.from_pretrained("KBlueLeaf/EQ-SDXL-VAE", cache_dir="../../models/vae").cuda().to(torch.bfloat16)


In [5]:
ds = load_from_disk("imagenet1k_eqsdxlvae_latents")
ds = iter(ds)

In [43]:
row = next(ds)
lats = row["vae_latent"]

with torch.no_grad():
    lats = torch.tensor(lats, dtype=torch.uint8)

    lats = lats.view(torch.float8_e5m2).to("cuda", torch.bfloat16).unsqueeze(0)
    image = vae.decode(lats).sample.to("cpu", torch.float32)

    print(lats.shape)
    print(image.shape)

    image = image.clamp(-1, 1)
    image = image.cpu().numpy()
    image = (image * 0.5 + 0.5) * 255
    image = image.astype(np.uint8)
    image = image.squeeze(0)  # Remove batch dimension
    image = image.transpose(1, 2, 0)  # Change from CHW to HWC format

    image = Image.fromarray(image)
    image.show()


torch.Size([1, 4, 28, 36])
torch.Size([1, 3, 224, 288])
